# UTILS

## code

In [ ]:
# coding: utf-8
"""Fonctions assez génériques pour être utilisées dans tout le projet."""
import math
import dateutil
import statistics as stat
from enum import Enum

from numpy import datetime64
import numpy as np


SEUIL_CATEGORIE_PAR_DEFAUT = 5


__all__ = [ 'Nature', 'valeur_numpy_nulle', 'donner_type', 'sigmoide', 'inverse_sigmoide' ]


class EnumManager(Enum):
    """Add-ons on Enum objects."""

    @classmethod
    def has_name(cls, name):
        """Check if the name is in the enumeration. """
        return name in cls._member_names_

    @classmethod
    def has_value(cls, value):
        """Check if the value is in the enumeration. """
        return value in cls._value2member_map_

    @classmethod
    def generate_name_from_value(cls, element, has_default_value=True):
        """Ensure coherency in the enumeration """
        # default name
        name = cls._member_names_.keys()[0] if has_default_value else None
        if cls.has_name(element):
            name = element
        elif cls.has_value(element):
            name = cls(element).name
        else:
            pass#print("Beware invalid role value. How is it possible?")
        return name


class Nature(EnumManager):
    """Enumerations of allowed data types."""
    CATEGORY = "Category"
    BINARY = "Binary"
    ORDERED_CATEGORY = "Ordered Category"
    UNORDERED_CATEGORY = "Unordered Category"
    NUMBER = "Number"
    DATETIME = "Datetime"


def valeur_numpy_nulle(variable):
    """ Pour savoir si une variable est NaN, il y a la fonction np.nan.
    Toutefois, elle ne fonctionne que sur des valeurs numériques.
    Cette fonction s'utilise sur tout type de fonction.
        
    Arguments d'entrées:
        variable (Python Object)
        
    Arguments de sorties:
        (bool)
    """
    return variable in [None, ""] or str(variable).lower() == "nan"

In [59]:
# type cas datetime
# https://stackoverflow.com/questions/25341945/check-if-string-has-date-any-format
# https://docs.python.org/fr/3.7/library/datetime.html
# https://stackoverflow.com/questions/13703720/converting-between-datetime-timestamp-and-datetime64
from dateutil.parser import parse, parserinfo 

class CustomParserInfo(parserinfo):
    """ TODO: real custom of date please"""
    
    MONTHS= [('Jan', 'January'), ('Feb', 'February'), ('Mar', 'March'), ('Apr', 'April'), ('May', 'May'), ('Jun', 'June'), ('Jul', 'July'), ('Aug', 'August'), ('Sep', 'Sept', 'September'), ('Oct', 'October'), ('Nov', 'November'), ('Dec', 'December')]

    WEEKDAYS= [('Mon', 'Lundi', 'Monday'), ('Tue', 'Tuesday'), ('Wed', 'Wednesday'), ('Thu', 'Thursday'), ('Fri', 'Friday'), ('Sat', 'Saturday'), ('Sun', 'Sunday')]
    
    @classmethod
    def from_language(cls, language=None, kwargs=None):
        return cls

DATETIME_DTYPE = [np.dtype('datetime64'), np.dtype('datetime64[s]'), np.dtype('datetime64[ns]') ]

    
def est_numpy_date(element):#, string, fuzzy=False, language='eng', kwargs=None):
    return any( y == date_type for date_type in DATETIME_DTYPE )

    
def est_date(element, str_kwargs=None):#, string, fuzzy=False, language='eng', kwargs=None):
    """
    """
    type_datetime = 
    type_series = isinstance(element, pd.Series)
    type_str = isinstance(element, str)
    type_list = isinstance(element, list)
    
    if type_datetime:
        return True
    
    if type_series:
        type_object = element.dtype == np.object
        if type_object:
            return any(map(est_numpy_date, element))
        return est_numpy_date(element)
    
    if type_str:
        return est_string_date(**{**(str_kwargs or {}), **{'string': element}})
    
    if type_list:
        return any(map(est_date, element))
        
        
    
    
    
    
    

def est_date(string, fuzzy=False, language='eng', kwargs=None):

y = pd.to_datetime(series).dtype
z = ndate(y)
a = y == np.datetime64
b = est_date(series[0], fuzzy=False, language=None, kwargs=None)
c = isinstance(series[0], (dt.timedelta, dt.date))






def est_date(string, fuzzy=False, language='eng', kwargs=None):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    parserinfo_instance = CustomParserInfo.from_language(language, kwargs)()
    try: 
        parse(string, fuzzy=fuzzy, parserinfo=parserinfo_instance)
        return True

    except:
        return False

NameError: name 'est_date' is not defined

In [25]:
def trouver_type(variable, seuil_categorie):
    """Déterminer le type d'une variable par calcul.

    Arguments d'entrées:
        variable (NumPy.array)
        seuil_categorie (int): valeur qui détermine le type d'une variable
            (numérique ou catégorique) suivant le nombre de valeurs distinctes.

    Arguments de sorties:
        (Enum)
    """
    nombre_de_valeurs_distinctes = len(set(variable))
    fuzzy=False
    language='eng'
    kwargs=None

    type_variable_categorie = variable.dtype == 'object'
    type_variable_datetime = variable.dtype == datetime64 or any(map(lambda s: is_date(s, fuzzy=fuzzy, language=language, kwargs=kwargs), variable))

    if type_variable_datetime:
        return Nature.DATETIME
    if nombre_de_valeurs_distinctes == 2:
        return Nature.BINARY
    elif nombre_de_valeurs_distinctes < seuil_categorie or type_variable_categorie:
        return Nature.CATEGORY

    else:
        return Nature.NUMBER


def donner_type(variable, seuil_categorie=None, type_attitre=None):
    """Retourner le type d'une variable.

    Arguments d'entrées:
        variable (NumPy.array)
        seuil_categorie (int): valeur qui détermine le type d'une variable
            (numérique ou catégorique) suivant le nombre de valeurs distinctes.
        type_attitre (str): possible type donnée par l'utilisateur à vérifier.

    Arguments de sorties:
        (Enum)
    """
    if seuil_categorie is None:
        seuil_categorie = SEUIL_CATEGORIE_PAR_DEFAUT
    
    type_attitre = Nature.generate_name_from_value(type_attitre, has_default_value=False)
    if type_attitre is None:
        return trouver_type(variable, seuil_categorie)
    return type_attitre


def sigmoide(x, valeur_sup=1.0, valeur_inf=.0, pente=1):
    """Personnalisable sigmoide.

    Arguments d'entrées:
        x (float)
        valeur_sup, valeur_inf, pente (float): paramètres
    
    Arguments de sorties:
        (float)
    """
    return valeur_inf + ( valeur_sup - valeur_inf ) / ( 1 + math.exp(- pente * x) )


def inverse_sigmoide(x, valeur_sup=1.0, valeur_inf=.0, pente=1):
    """Personnalisable inverse sigmoide.

    Arguments d'entrées:
        x (float)
        valeur_sup, valeur_inf, pente (float): paramètres
    
    Arguments de sorties:
        (float)
    """
    return math.log( ( x - valeur_inf ) / ( valeur_sup - valeur_inf) ) / pente

## code

In [2]:
import pandas as pd
import numpy as np
import os
import operator
import datetime as dt

In [3]:
dataset = pd.read_csv("titanic.csv")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887 entries, 0 to 886
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Survived                 887 non-null    int64  
 1   Pclass                   887 non-null    int64  
 2   Name                     887 non-null    object 
 3   Sex                      887 non-null    object 
 4   Age                      887 non-null    float64
 5   Siblings/Spouses Aboard  887 non-null    int64  
 6   Parents/Children Aboard  887 non-null    int64  
 7   Fare                     887 non-null    float64
dtypes: float64(2), int64(4), object(2)
memory usage: 55.6+ KB


In [4]:
age = dataset.Age
fare = dataset.Fare
survived = dataset.Survived
sex = dataset.Sex

In [5]:
{ k: trouver_type(dataset[k], SEUIL_CATEGORIE_PAR_DEFAUT) for k in dataset }

{'Survived': <Nature.BINARY: 'Binary'>,
 'Pclass': <Nature.CATEGORY: 'Category'>,
 'Name': <Nature.CATEGORY: 'Category'>,
 'Sex': <Nature.BINARY: 'Binary'>,
 'Age': <Nature.NUMBER: 'Number'>,
 'Siblings/Spouses Aboard': <Nature.NUMBER: 'Number'>,
 'Parents/Children Aboard': <Nature.NUMBER: 'Number'>,
 'Fare': <Nature.NUMBER: 'Number'>}

In [29]:
series = pd.Series([dt.date(2020, 8, i) for i in range(1, 12)])
series

0     2020-08-01
1     2020-08-02
2     2020-08-03
3     2020-08-04
4     2020-08-05
5     2020-08-06
6     2020-08-07
7     2020-08-08
8     2020-08-09
9     2020-08-10
10    2020-08-11
dtype: object

In [62]:
series.dtype == np.object

True

In [56]:
y = pd.to_datetime(series).dtype
ndate = lambda y: any( y == t for t in (np.dtype('datetime64'), np.dtype('datetime64[s]'), np.dtype('datetime64[ns]')) )
ndate(y)

True

In [57]:
# type_variable_datetime = variable.dtype == datetime64 or any(map(lambda s: is_date(s, fuzzy=fuzzy, language=language, kwargs=kwargs), variable))
y = pd.to_datetime(series).dtype
z = ndate(y)
a = y == np.datetime64
b = is_date(series[0], fuzzy=False, language=None, kwargs=None)
c = isinstance(series[0], (dt.timedelta, dt.date))
y, z, a, b, c

(dtype('<M8[ns]'), True, False, False, True)

In [12]:

trouver_type(series, SEUIL_CATEGORIE_PAR_DEFAUT)

<Nature.CATEGORY: 'Category'>

# CORRELATION

In [5]:
# coding: utf-8
"""
    Fonctions permettant de calculer les différentes corrélations
    entre les variables d'un même jeu de données.
    Un vecteur peut être une liste, un Pandas.Series ou NumPy.array. 
"""
import math
import statistics as stat
import operator

from scipy import stats
from numpy import datetime64
import numpy as np
import pandas as pd

#from .utils import *


def calculer_covariance(variable_1, variable_2):
    """Calcul de la covariance empirique entre deux variables quantitatives.

    Arguments d'entrée:
        variable_1, variable_2 (numpy.array)

    Arguments de sortie:
        (float)
    """
    n = len(variable_1)
    moyenne_variable_1 = stat.mean(variable_1)
    moyenne_variable_2 = stat.mean(variable_2)
    return 0 if n == 0 else sum(x_i_1 * x_i_2 for x_i_1, x_i_2 in zip(variable_1 - moyenne_variable_1, variable_2 - moyenne_variable_2)) / n


def calculer_correlation_pearson(variable_1, variable_2):
    """Calcul de la correlation entre deux variables quantitatives.
    On retourne donc une valeur entrez 0 et 1.

    Arguments d'entrée:
        variable_1, variable_2 (numpy.array)

    Arguments de sortie:
        (float)
    """
    correlation = calculer_covariance(variable_1, variable_2)
    variance_1 = calculer_covariance(variable_1, variable_1)
    variance_2 = calculer_covariance(variable_2, variable_2)

    if variance_1 == 0 or variance_2 == 0:
        return 0
    else:
        return correlation / (math.sqrt(variance_1) * math.sqrt(variance_2))


def test_correlation_pearson(variable_1, variable_2, covariance=None):
    """Test de corrélation de Pearson
    pour une corrélation entre deux variables quantitatives.

    Arguments d'entrées:
        variable_1, variable_2 (NumPy.array)
        covariance (Python function): function pour calculer une covariance entre deux variables.

    Arguments de sorties:
        statistique, p_valeur, coefficient (float)
    """
    if covariance is None:
        covariance = calculer_covariance

    # On supprime les valeurs nulles si nécessaire
    condition = ~( np.isnan(variable_1) | np.isnan(variable_2) )
    variable_1, variable_2 = variable_1[condition], variable_2[condition]

    n = len(variable_1)

    # Calcul du coefficient
    coefficient = calculer_correlation_pearson(variable_1, variable_2)

    # Calcul de la statistique
    statistique = np.inf
    if coefficient != 1.0:
        statistique = coefficient / math.sqrt((1 - coefficient ** 2)/(n - 2))

    # Calcul de la p_valeur
    p_valeur = (1 - stats.t.cdf(x=statistique, df=n - 2))

    return statistique, p_valeur, coefficient


def test_correlation_eta_squared(variable_quantitative, variable_qualitative):
    """Test de corrélation de eta squared
    pour une corrélation entre une variable quantitative et une variable qualitative.

    Arguments d'entrées:
        variable_quantitative, variable_qualitative (NumPy.array)

    Arguments de sorties:
        statistique, p_valeur, coefficient (float)
    """
    # On supprime les valeurs nulles si nécessaire
    variable_qualitative = variable_qualitative.astype(str)

    condition = ~( np.isnan(variable_quantitative) | np.isnan([ valeur_numpy_nulle(x) for x in variable_qualitative ]) )
    variable_quantitative = variable_quantitative[condition]
    variable_qualitative = variable_qualitative[condition]

    n_quantitative = len(variable_quantitative)
    classes = np.unique(variable_qualitative)
    n_qualitative = len(classes)

    # Calcul du coefficient
    moyenne_quantitative = stat.mean(variable_quantitative)
    classes = [ variable_quantitative[variable_qualitative == cl] for cl in classes ]
    classes = [(len(vecteur), stat.mean(vecteur)) for vecteur in classes]
    coefficient_sct = sum((valeur - moyenne_quantitative)
                          ** 2 for valeur in variable_quantitative)
    coefficient_sce = sum(nombre_classe_i * (moyenne_classe_i - moyenne_quantitative)
                          ** 2 for nombre_classe_i, moyenne_classe_i in classes)

    coefficient = coefficient_sce / coefficient_sct if coefficient_sct != 0 else np.inf

    # Calcul de la statistique
    statistique = (coefficient * (n_quantitative - n_qualitative)
                   ) / ((1 - coefficient) * (n_qualitative - 1))

    # Calcul de la p_valeur
    p_valeur = 1 - stats.f.cdf(statistique, n_quantitative -
                              n_qualitative, n_qualitative - 1)

    return statistique, p_valeur, coefficient


def calculer_tableau_contingence(variable_1, variable_2, nom_1=None, nom_2=None, rajouter_colonne_total=False):
    """ Calcul du tableau de contingence entre deux variables qualitatives.
    Basé sur https://openclassrooms.com/fr/courses/4525266-decrivez-et-nettoyez-votre-jeu-de-donnees/4775616-analysez-deux-variables-qualitatives-avec-le-chi-2


    Arguments d'entrées:
        variable_1, variable_2 (numpy.array)
        nom_1, nom_2 (str)

    Arguments de sortie:
        tableau_de_contingence (NumPy.array) 
    """
    if nom_1 is None:
        nom_1 = 'variable_1'
    
    if nom_2 is None:
        nom_2 = 'variable_2'
    
    
    nom_total_1, nom_total_2 = 'total_1', 'total_2'
    tableau_de_contingence = (pd.DataFrame(data={nom_1: variable_1, nom_2: variable_2})
                              .pivot_table(index=nom_1, columns=nom_2, aggfunc=len))

    if rajouter_colonne_total:
        tableau_de_contingence.loc[:,
                                   nom_total_2] = tableau_de_contingence[nom_1].value_counts()
        tableau_de_contingence.loc[nom_total_1,
                                   :] = tableau_de_contingence[nom_2].value_counts()
        tableau_de_contingence.loc[nom_total_1,
                                   nom_total_2] = len(variable_1)

    return tableau_de_contingence.fillna(0).values


def test_correlation_chi_squared(variable_1, variable_2, nom_1=None, nom_2=None, contingence=None, **kwargs):
    """Test de corrélation du chi-deux
    pour une corrélation entre deux variables qualtitatives.

    Arguments d'entrées:
        variable_1, variable_2 (NumPy.array)
        nom_1, nom_2 (str)
        contingence (Python function): function pour calculer un tableau de contingence entre deux variables.

    Arguments de sorties:
        statistique, p_valeur, coefficient (float): coefficient basé sur le coefficient de Cramer 
    """
    if contingence is None:
        contingence = lambda var_1, var_2: calculer_tableau_contingence(variable_1=var_1,
                                                                        variable_2=var_2,
                                                                        nom_1=nom_1,
                                                                        nom_2=nom_2,
                                                                        rajouter_colonne_total=False)

    # On supprime les valeurs nulles si nécessaire
    variable_1 = variable_1.astype(str)
    variable_2 = variable_2.astype(str)
    condition = ~( np.isnan([ valeur_numpy_nulle(x) for x in variable_1]) | np.isnan([ valeur_numpy_nulle(x) for x in variable_2]) )
    variable_1 = variable_1[condition]
    variable_2 = variable_2[condition]

    n = len(variable_1)
    tableau_de_contingence = contingence(variable_1, variable_2)
    total_1 = pd.Series(variable_1).value_counts()
    total_2 = pd.Series(variable_2).value_counts()

    terme_independance = sum( t_1 * t_2 for t_1, t_2 in zip(total_1, total_2) ) / n

    # Calcul de la statistique
    statistique = ( (tableau_de_contingence - terme_independance) ** 2 / terme_independance ).tolist()
    statistique = sum( sum(ligne) for ligne in statistique )              

    # Calcul du coefficient
    coefficient = math.sqrt( statistique / (n * (min(len(total_1), len(total_2)) - 1)) )

    # Calcul de la p_valeur
    p_valeur = 1 - stats.chi2.cdf(x=statistique, df=(len(total_1) - 1) * (len(total_2) - 1))

    return statistique, p_valeur, coefficient


def test_de_correlation(variable_1, variable_2, nom_1=None, nom_2=None, seuil_categorie=None, *args, **kwargs):
    """Pour calculer la correlation variables.

    Arguments d'entrée:
        variable_1, variable_2 (pandas.Series)
        nom_1, nom_2 (str)

    Arguments de sortie:
        statistique, p_valeur, coefficient (float)
    """
    variable_1_type = donner_type(variable=variable_1, seuil_categorie=seuil_categorie, type_attitre=kwargs.get(nom_1, None))
    variable_2_type = donner_type(variable=variable_2, seuil_categorie=seuil_categorie, type_attitre=kwargs.get(nom_2, None))


    if (variable_1_type, variable_2_type) == (Nature.NUMBER, Nature.NUMBER):
        covariance = kwargs.get('covariance', None)
        return test_correlation_pearson(variable_1, variable_2, covariance)
    elif variable_1_type == Nature.NUMBER:
        return test_correlation_eta_squared(variable_1, variable_2)
    elif variable_2_type == Nature.NUMBER:
        return test_correlation_eta_squared(variable_2, variable_1)
    else:
        contingence = kwargs.get('contingence', None)
        return test_correlation_chi_squared(variable_1=variable_1,
                                            variable_2=variable_2,
                                            nom_1=nom_1,
                                            nom_2=nom_2,
                                            contingence=contingence
                                            ) 


def matrice_de_correlation(tableau, format_compact=True, calculer_autocorrelation=False, retourner_liste=False, *args, **kwargs):
    """Pour calculer la matrice de correlation d'un jeu de données.

    Arguments d'entrée:
        tableau (Pandas.DataFrame)
        format_compact (bool): Si Vrai, matrice de triplet sinon trois matrices en sortie
        calculer_autocorrelation (bool): Si Vrai, calculer l'autocorrelation qui doit s'annuler
            #TO FIX à supprimer, peut-être inutile
        retourner_liste (bool): Si vrai retourner une liste
            #TO FIX à supprimer, peut-être inutile

    Arguments de sortie:
        correlation or correlation_statistique, correlation_p_valeur, correlation_coefficient (NumPy.array or list)
    """
    nom_de_variables = list(tableau)
    correlation = []
    for variable_1 in nom_de_variables:
        correlation_pour_variable_1 = []
        for variable_2 in nom_de_variables:
            #print()
            #print(variable_1, " vs ", variable_2)
            triplet = (np.nan, np.nan, np.nan)
            if not calculer_autocorrelation and variable_1 == variable_2:
                triplet = (1, np.nan, 1)
            else:
                triplet = test_de_correlation(tableau[variable_1].values,
                                              tableau[variable_2].values,
                                              *args,
                                              **kwargs)
            correlation_pour_variable_1.append(triplet)
            #print("***********")
        correlation.append(correlation_pour_variable_1)

    if format_compact:
        if not retourner_liste:
            correlation = np.array(correlation)
        return correlation
    else:
        correlation_statistique = [list(map(operator.itemgetter(0), ligne)) for ligne in correlation]
        correlation_p_valeur = [list(map(operator.itemgetter(1), ligne)) for ligne in correlation]
        correlation_coefficient = [list(map(operator.itemgetter(2), ligne)) for ligne in correlation]

        if not retourner_liste:
            correlation_statistique = np.array(correlation_statistique)
            correlation_p_valeur = np.array(correlation_p_valeur)
            correlation_coefficient = np.array(correlation_coefficient)
        return correlation_statistique, correlation_p_valeur, correlation_coefficient


def normaliser_significativite_booleen(p_valeur, alpha, tolerance, *args, **kwargs):
    """Retourne une valeur booléenne pour qualifier la significativité d'un test statistique.

    Arguments d'entrée:
        p_valeur, alpha, tolerance (float)


    Arguments de sortie:
        significativite (bool)
    """
    significativite = p_valeur >= alpha
    return significativite


def normaliser_significativite_numerique(p_valeur, alpha, tolerance, *args, **kwargs):
    """Retourne une valeur entre 0 et 1 pour quantifier la significativité d'un test statistique.

    Arguments d'entrée:
        p_valeur, alpha, tolerance (float)


    Arguments de sortie:
        significativite (float)
    """
    # Première normalisation entre 0 et 1 
    significativite = max(0, (p_valeur - alpha) / (1 - alpha))

    # Seconde normalisation pour changer la répartition des valeurs normalisées
    significativite = sigmoide(x=significativite,
                               valeur_sup=kwargs.get('valeur_sup',1.0),
                               valeur_inf=kwargs.get('valeur_inf',.0),
                               pente=kwargs.get('pente',1))
    return significativite


def matrice_de_significativite(tableau, alpha, retourner_booleen=False, tolerance=.01, *args, **kwargs):
    """Retourne une matrice où chaque valeur sera comprise entre 0 et 1 ou sera booléenne.

    Arguments d'entrée:
            correlation_p_valeur (Numpy.array or list)
            alpha (float)
            retourner_booleen (bool)
            tolerance (float)


        Arguments de sortie:
            tableau_de_taille (Numpy.array)
    """
    to_array = isinstance(tableau, np.ndarray)
    if to_array:
        tableau = tableau.tolist()
    
    normaliser_significativite = None
    if retourner_booleen:
        normaliser_significativite = lambda s: normaliser_significativite_booleen(p_valeur=s,
                                                                                  alpha=alpha,
                                                                                  tolerance=tolerance,
                                                                                  *args, 
                                                                                  **kwargs)
    else:
        normaliser_kwargs = { 'valeur_sup': 1.0,
                              'valeur_inf': tolerance,
                              'pente': 5
                            }
        normaliser_significativite = lambda s: normaliser_significativite_numerique(p_valeur=s,
                                                                                    alpha=alpha,
                                                                                    tolerance=tolerance,
                                                                                    *args, 
                                                                                    **{**kwargs, **normaliser_kwargs})

    tableau_de_taille = [list(map(normaliser_significativite, ligne)) for ligne in tableau]

    if to_array:
        return np.array(tableau_de_taille)
    return tableau_de_taille


ImportError: attempted relative import with no known parent package

# TRANSFORMATION

In [ ]:
# coding: utf-8
"""Module centré sur le preprocessing d'un jeu de données.
Bien que basé sur un projet d'analyse de données spécifique (OpenFoodFacts),
le contenu a pour vocation d'être générique."""
import math
import time

import numpy as np
import pandas as pd


def eliminer_colonne_vide(tableau, taux_de_vide_minimum=0.50, retourner_details=True, chemin_fichier='barh_classement_colonnes_vides.png'):
    """Supprimer les colonnes avec un taux de remplissage inférieur à une référence donnée.
    
    Arguments d'entrée:
        tableau (pandas.DataFrame)
        taux_de_vide_minimum (float): au delà de ce taux, la colonne sera considérée vide
        retourner_details (bool): si True, renvoie le classement des colonnes
            sur leurs taux de remplissage.
    
    Arguments de sortie:
        tableau_nettoye (pandas.DataFrame)
        classement (pandas.DataFrame)
    """
    classement = pd.DataFrame()
    classement['colonne'] = list(tableau)
    classement['taux_de_vide'] = (
        classement['colonne']
        .apply(lambda colonne: tableau[colonne].isna().mean())
    )
    classement = classement.sort_values('taux_de_vide', ascending=False)
    print("clst df:\n ", classement)
    
    colonnes_a_eliminer = classement.loc[classement['taux_de_vide']>=taux_de_vide_minimum, 'colonne']
    tableau_nettoye = tableau.drop(columns=colonnes_a_eliminer)

    if retourner_details:
        # Initialise plot
        ax = classement.set_index('colonne').plot(kind='barh', figsize=(8, 10), color='#86bf91', zorder=2, width=0.85)

        # Despine -- remove figure's borders
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)

        # Set x-axis
        ax.set_xlabel("Ratio d'éléments vides", labelpad=20, weight='bold', size=12)
        # il faudra voir

        # Set y-axis
        ax.set_ylabel("Colonnes", labelpad=20, weight='bold', size=12)

        # Add a legend
        ax.legend(ncol=2, loc="upper right")


        # Save figure
        (ax
        .get_figure()
        .savefig(chemin_fichier, bbox_inches="tight")
        )

        return tableau_nettoye, classement
    
    return tableau_nettoye


def eliminer_ligne_vide(tableau, taux=None, retourner_details=True):
    """Supprimer les lignes avec un taux de remplissage inférieur à une référence donnée.
    
    Arguments d'entrée:
        tableau (pandas.DataFrame)
        taux (float)
        retourner_details (bool): si True, renvoie le classement des colonnes
            sur leurs taux de remplissage.
    
    Arguments de sorties:
        tableau_nettoye (pandas.DataFrame)
        classement (pandas.DataFrame)
    """
    if not taux:
        taux = 0.50
    taux = len(tableau.columns) * taux // 1

    lignes_a_eliminer = tableau.isna().sum(1)
    tableau_nettoye = tableau[ lignes_a_eliminer>=taux ]

    if retourner_details:
        pass
        # (classement
        # .plot(kind='barh')
        # .get_figure()
        # .savefig(chemin_fichier)
        # )
    return tableau_nettoye


def trouver_valeur_par_défaut(valeur_par_defaut, clef):
    """Permet de personnaliser une valeur par défaut suivant un axe."""
    if isinstance(valeur_par_defaut, dict):
        return valeur_par_defaut.get(clef, None)
    return valeur_par_defaut


def premiere_occurence(vecteur, valeur_par_defaut=None):
    """Wrapper de la méthode first_valid_index d'un Pandas.Series.
    
    Arguments d'entrée:
        vecteur (pandas.Series)
        valeur_par_defaut
    Arguments de sortie:
        (Python Object)
    """
    index = vecteur.first_valid_index()
    if index is None:
        return valeur_par_defaut
    return vecteur[index]


def plus_frequente_occurence(vecteur, valeur_par_defaut=None):
    """Wrapper de la méthode first_valid_index d'un Pandas.Series.
    
    Arguments d'entrée:
        vecteur (pandas.Series)
        valeur_par_defaut
    Arguments de sortie:
        (Python Object)
    """
    return vecteur.value_counts(ascending=False, dropna=False).iloc[0]


def calculer_imputation(tableau, strategie='première occurence', valeur_par_defaut=None):
    """Imputer suivant la strategie choisie.
    
    Arguments d'entrée:
        tableau (pandas.DataFrame)
        strategie (str):
            'première occurence' retourne la première valeur non nulle,
            'la plus fréquente' retourne la valeur la plus fréquente non nulle
    
    Arguments de sortie:
        tableau_nettoye (pandas.DataFrame)
    """
    fonction = None
    if strategie =="première occurence":
        fonction = premiere_occurence
    elif strategie=="la plus fréquente":
        fonction = plus_frequente_occurence
    else:
        raise ValueError(f"l'argument strategie ne peut prendre que les valeurs suivantes: 'première occurence' et 'la plus fréquente'. La valeur donnée ici est {strategie}")
    if tableau.empty:
        raise ValueError("l'argument tableau est vide.")
    tableau_impute = {index: fonction(tableau[index], trouver_valeur_par_défaut(valeur_par_defaut, index)) for index in tableau.columns}
    tableau_impute = pd.Series(tableau_impute)
    return tableau_impute


def eliminer_doublons(tableau, colonnes_ciblees=None, strategie='première occurence'):
    """Retourne un tableau avec des individus uniques
    basés sur les colonnes ciblées.
    
    Arguments d'entrée:
        tableau (pandas.DataFrame)
        colonnes_ciblees (list)
        strategie (str): méthode pour trouver l'unique individu
    
    Arguments de sortie:
        tableau_nettoye (pandas.DataFrame)
    """
    if colonnes_ciblees is None or not isinstance(colonnes_ciblees, list):
        if not isinstance(colonnes_ciblees, list):
            print(f"l'argument colonnes_ciblees n'est pas de type list mais {type(colonnes_ciblees)}")
            return tableau.drop_duplicates()
    methode = lambda tableau: calculer_imputation(tableau, strategie)
    return (
        tableau
        .groupby(colonnes_ciblees)
        .agg(methode)
        .reset_index(drop=True)
    )


# PLOT

In [3]:
# coding: utf-8
"""Module centré sur le nettoyage et l'analyse exploratoire d'un jeu de données.
Bien que basé sur un projet d'analyse de données spécifique (OpenFoodFacts),
le contenu a pour vocation d'être générale."""
import time

import matplotlib.pyplot as plt
import seaborn as sns



def dessiner_graphique_barres(tableau, colonne, fichier, valeur_manquante='valeur manquante', titre=None, rotation=None):
    """Implémenter un histogramme pour une variable qualitative.
    
    Arguments d'entrée:
        tableau (pandas.DataFrame)
        colonne (str)
        fichier (str): chemin relatif du graphique
        valeur_manquante (str): alias pour les np.nan
        titre (str)
        rotation (int)
    """
    if titre is None:
        titre = f"Histogramme de la variable qualitative {colonne}"
    denombrement = (
        tableau
        [colonne]
        .value_counts(normalize=True, dropna=False)
        .mul(100)
        .reset_index()
        .rename(columns={'index': colonne, colonne: 'pourcentage'})
        .fillna(valeur_manquante)
    )
    barres = sns.barplot(x=colonne, y="pourcentage", data=denombrement)
    for barre in barres.patches:
        barres.annotate('{:.2f}%'.format(barre.get_height()), (barre.get_x()+0.5, barre.get_height()+1))
    plt.title(titre)
    plt.xlabel(colonne)
    plt.ylabel('pourcentage [%]')
    if rotation:
        plt.setp(barres.get_xticklabels(), rotation=rotation)
    (barres
    .get_figure()
    .savefig(fichier, bbox_inches="tight"))
    plt.close()


def dessiner_distributions_univariees(tableau, nbr_colonnes, fichier, titre=None, dropna=True):
    """Implémenter un histogramme ou une distribution pour chaque variable.
    
    Arguments d'entrée:
        tableau (pandas.DataFrame)
        nbr_colonne (int): on aura une table de graphique de nbr_colonnes colonnes
        fichier (str): chemin relatif du graphique
        titre (str)
    """
    if titre is None:
        titre = f"Distributions univariées"
    nbr_graphiques = len(tableau.columns)
    fig = plt.figure(figsize=(nbr_graphiques, nbr_graphiques))
    for index, colonne in enumerate(tableau.columns):
        vecteur = tableau[colonne]
        if dropna:
            vecteur = vecteur.dropna()
        axe = fig.add_subplot((nbr_graphiques // nbr_colonnes) + 1,
                            nbr_colonnes,
                            index + 1)
        sns.distplot(vecteur,
                    ax=axe,
                    hist=False,
                    #rug=True,
                    kde_kws={"kernel": "gau"}
        )
    plt.axis('tight')
    plt.tight_layout()
    plt.gcf().savefig(fichier, bbox_inches="tight")
    plt.close()


def dessiner_graphique_donut(tableau, colonne, fichier, valeur_manquante='valeur manquante', titre=None):
    """Implémenter un donut pour une variable qualitative.
    
    Arguments d'entrée:
        tableau (pandas.DataFrame)
        colonne (str)
        fichier (str): chemin relatif du graphique
        valeur_manquante (str): alias pour les np.nan
        titre (str)
    """
    if titre is None:
        titre = f"Répartition de la variable qualitative {colonne}"
    camembert = (
        tableau
        [colonne]
        .value_counts(normalize=True, dropna=False)
        .mul(100)
        .reset_index()
        .rename(columns={'index': colonne, colonne: 'pourcentage'})
        .fillna(valeur_manquante)
    )
    nbr_categories, _ = camembert.shape
    if colormap is None:
        #colormap = plt.get_cmap('tab20')
        colormap = sns.color_palette("husl", nbr_categories)
    # pour isoler la première part
    explode = [0.1] + [0] * (nbr_categories - 1)
    labels = ["{}: {:.2f}%".format(vecteur[colonne], vecteur['pourcentage']) for _, vecteur in camembert.iterrows()]
    plt.pie(camembert['pourcentage'].mul(1/100),
            labels=labels,
            colors=colormap,
            explode=explode,
        )
    trou_du_donut = plt.Circle(xy=(0, 0), radius=0.47, facecolor='white')
    plt.gcf().gca().add_artist(trou_du_donut)
    plt.axis('equal')
    plt.gcf().savefig(fichier, bbox_inches="tight")
    plt.close()



#f, ax = plt.subplots(figsize=(22,5))
#cols = [x for x in df]
#idx = cols.index('nutrition-score-fr_100g')
#sns.heatmap(corr_mat[idx, :].reshape(1, -1),
#            yticklabels='',
#            xticklabels=cols, square=True, linewidths=1, linecolor='white',
#            cbar=True, cbar_kws={"shrink": .45},
#            annot=True, fmt='.3f',
#            cmap="BuPu"
#           );
#plt.title("Corrélation de la variable 'nutrition-score-fr_100g'");
#plt.gcf().savefig('graphs/my_heatmap_score_first.png', bbox_inches="tight")



# HEATMAP

In [4]:
"""
    Functions to visualize matrices of data.
    It is a custom version of a Heatmap allowing
    cells size's customization.
    It is based on matrix.py in https://github.com/mwaskom/seaborn
    ( commit 065d3c1 ) by Michael L. Waskom .
"""

from __future__ import division
#import itertools
import functools
#import datetime
import numbers
import operator

import matplotlib as mpl
#from matplotlib.collections import LineCollection
import matplotlib.pyplot as plt
#from matplotlib import gridspec
#import matplotlib.patheffects as patheffects
import numpy as np
import pandas as pd
#from scipy.cluster import hierarchy

import seaborn as sns
from seaborn import cm
#from seaborn.axisgrid import Grid
from seaborn.utils import (
    despine, axis_ticklabels_overlap, relative_luminance, to_utf8)
from seaborn.external.six import string_types

__all__  = [ 'custom_cells_heatmap' ]


def _replace_bool(value):
    """NEW-- Custom function to replace boolean values"""
    if not isinstance(value, bool):
        return value
    return 1.0 if value else 0.0


def _index_to_label(index):
    """Convert a pandas index or multiindex to an axis label."""
    if isinstance(index, pd.MultiIndex):
        return "-".join(map(to_utf8, index.names))
    else:
        return index.name


def _index_to_ticklabels(index):
    """Convert a pandas index or multiindex into ticklabels."""
    if isinstance(index, pd.MultiIndex):
        return ["-".join(map(to_utf8, i)) for i in index.values]
    else:
        return index.values


def _convert_colors(colors):
    """Convert either a list of colors or nested lists of colors to RGB."""
    to_rgb = mpl.colors.colorConverter.to_rgb

    if isinstance(colors, pd.DataFrame):
        # Convert dataframe
        return pd.DataFrame({col: colors[col].map(to_rgb)
                             for col in colors})
    elif isinstance(colors, pd.Series):
        return colors.map(to_rgb)
    else:
        try:
            to_rgb(colors[0])
            # If this works, there is only one level of colors
            return list(map(to_rgb, colors))
        except ValueError:
            # If we get here, we have nested lists
            return [list(map(to_rgb, l)) for l in colors]


def _matrix_mask(data, mask):
    """
        Ensure that data and mask are compatible and add missing values.
        Values will be plotted for cells where ``mask`` is ``False``.
        ``data`` is expected to be a DataFrame; ``mask`` can be an array or
        a DataFrame.
    """
    if mask is None:
        mask = np.zeros(data.shape, np.bool)

    if isinstance(mask, np.ndarray):
        # For array masks, ensure that shape matches data then convert
        if mask.shape != data.shape:
            raise ValueError("Mask must have the same shape as data.")

        mask = pd.DataFrame(mask,
                            index=data.index,
                            columns=data.columns,
                            dtype=np.bool)

    elif isinstance(mask, pd.DataFrame):
        # For DataFrame masks, ensure that semantic labels match data
        if not mask.index.equals(data.index) \
           and mask.columns.equals(data.columns):
            err = "Mask must have the same index and columns as data."
            raise ValueError(err)

    # Add any cells with missing data to the mask
    # This works around an issue where `plt.pcolormesh` doesn't represent
    # missing data properly
    mask = mask | pd.isnull(data)

    return mask


def _normalize_cell_size(size, minimum, maximum, alias_true, alias_false, alias_nan):
    """NEW--"""
    if isinstance(size, bool):
        return alias_true if size else alias_false
    elif np.isnan(size):
        return alias_nan
    elif size <= minimum:
        return minimum
    elif size >= maximum:
        return maximum
    else:
        return size


class _CustomCellHeatMapper(object):
    """NEW--Custom version of _HeatMapper adding the control of the cell size."""

    DEFAULT_VMIN_CELLS = .1
    DEFAULT_VMAX_CELLS = 1

    def __init__(self, data, vmin, vmax, cmap, center, robust, annot, fmt,
                 annot_kws, cbar, cbar_kws, shape_kws,
                 data_cells, vmin_cells, vmax_cells, robust_cells, robust_type='percentile',
                 xticklabels=True, yticklabels=True, mask=None, normalize_cells=True):
        """
            Initialize the plotting object.
        """
        # NEW-- PLOT_DATA
        # We always want to have a DataFrame with semantic information
        # and an ndarray to pass to matplotlib
        if isinstance(data, pd.DataFrame):
            plot_data = data.values
        else:
            plot_data = np.asarray(data)
            data = pd.DataFrame(plot_data)


        # We always want to have a DataFrame with semantic information
        # and an ndarray to pass to matplotlib
        if data_cells is None:
            data_cells = pd.DataFrame(data=np.ones(data.shape, dtype=float),
                                      columns=data.columns,
                                      index=data.index)

        if isinstance(data_cells, pd.DataFrame):
            plot_cells = data_cells.values
        else:
            plot_cells = np.asarray(data_cells)
            data_cells = pd.DataFrame(plot_cells)

        
        # NEW-- PLOT_CELLS
        # Validate the mask and convert to DataFrame
        mask = _matrix_mask(data, mask)

        plot_data = np.ma.masked_where(np.asarray(mask), plot_data)
        plot_cells = np.ma.masked_where(np.asarray(mask), plot_cells)


        # Get good names for the rows and columns
        xtickevery = 1
        if isinstance(xticklabels, int):
            xtickevery = xticklabels
            xticklabels = _index_to_ticklabels(data.columns)
        elif xticklabels is True:
            xticklabels = _index_to_ticklabels(data.columns)
        elif xticklabels is False:
            xticklabels = []

        ytickevery = 1
        if isinstance(yticklabels, int):
            ytickevery = yticklabels
            yticklabels = _index_to_ticklabels(data.index)
        elif yticklabels is True:
            yticklabels = _index_to_ticklabels(data.index)
        elif yticklabels is False:
            yticklabels = []

        # Get the positions and used label for the ticks
        nx, ny = data.T.shape

        if not len(xticklabels):
            self.xticks = []
            self.xticklabels = []
        elif isinstance(xticklabels, string_types) and xticklabels == "auto":
            self.xticks = "auto"
            self.xticklabels = _index_to_ticklabels(data.columns)
        else:
            self.xticks, self.xticklabels = self._skip_ticks(xticklabels,
                                                             xtickevery)

        if not len(yticklabels):
            self.yticks = []
            self.yticklabels = []
        elif isinstance(yticklabels, string_types) and yticklabels == "auto":
            self.yticks = "auto"
            self.yticklabels = _index_to_ticklabels(data.index)
        else:
            self.yticks, self.yticklabels = self._skip_ticks(yticklabels,
                                                             ytickevery)

        # Get good names for the axis labels
        xlabel = _index_to_label(data.columns)
        ylabel = _index_to_label(data.index)
        self.xlabel = xlabel if xlabel is not None else ""
        self.ylabel = ylabel if ylabel is not None else ""

        # Determine good default values for the colormapping
        self._determine_cmap_params(plot_data, vmin, vmax,
                                    cmap, center, robust)

        # Determine good default values for the sizemapping according to a feature
        self._determine_cells_params(plot_cells, vmin_cells,
                                     vmax_cells, robust_cells,
                                     robust_type, normalize_cells)

        # Sort out the annotations
        if annot is None:
            annot = False
            annot_data = None
        elif isinstance(annot, bool):
            if annot:
                annot_data = plot_data
            else:
                annot_data = None
        else:
            try:
                annot_data = annot.values
            except AttributeError:
                annot_data = annot
            if annot.shape != plot_data.shape:
                raise ValueError('Data supplied to "annot" must be the same '
                                 'shape as the data to plot.')
            annot = True

        # Save other attributes to the object
        self.data = data
        self.plot_data = plot_data

        # NEW-- Save other attributes to the object
        self.data_cells = data_cells
        self.plot_cells = plot_cells

        self.annot = annot
        self.annot_data = annot_data

        self.fmt = fmt
        self.annot_kws = {} if annot_kws is None else annot_kws
        self.cbar = cbar
        self.cbar_kws = {} if cbar_kws is None else cbar_kws
        self.cbar_kws.setdefault('ticks', mpl.ticker.MaxNLocator(6))
        # NEW-- 
        self.shape_kws = {} if shape_kws is None else shape_kws

    def _determine_cmap_params(self, plot_data, vmin, vmax,
                               cmap, center, robust):
        """Use some heuristics to set good defaults for colorbar and range."""
        calc_data = plot_data.data[~np.isnan(plot_data.data)]
        if vmin is None:
            vmin = np.percentile(calc_data, 2) if robust else calc_data.min()
        if vmax is None:
            vmax = np.percentile(calc_data, 98) if robust else calc_data.max()
        self.vmin, self.vmax = vmin, vmax

        # Choose default colormaps if not provided
        if cmap is None:
            if center is None:
                self.cmap = cm.rocket
            else:
                self.cmap = cm.icefire
        elif isinstance(cmap, string_types):
            self.cmap = mpl.cm.get_cmap(cmap)
        elif isinstance(cmap, list):
            self.cmap = mpl.colors.ListedColormap(cmap)
        else:
            self.cmap = cmap

        # Recenter a divergent colormap
        if center is not None:
            vrange = max(vmax - center, center - vmin)
            normlize = mpl.colors.Normalize(center - vrange, center + vrange)
            cmin, cmax = normlize([vmin, vmax])
            cc = np.linspace(cmin, cmax, 256)
            self.cmap = mpl.colors.ListedColormap(self.cmap(cc))

    def _determine_cells_params(self, plot_cells, vmin_cells, vmax_cells, robust_cells, robust_type, normalize_cells):
        """ NEW-- Use some heuristics to set good defaults for cells' size."""
        # Handle unknown robust methods
        if robust_type not in [ 'percentile', 'boundary' ] and robust_cells:
            raise ValueError(f"Incorrect robust_type: {robust_type} instead of 'percentile' or 'boundary.")

        # Handle incorrect types (only accepted or np.bool and np.numeric)
        type_cells = [ list(map(type,l)) for l in plot_cells ]
        available_types = functools.reduce(operator.xor, map(set, type_cells))
        invalid_types = [ _type for _type in available_types if not issubclass(_type, (bool, numbers.Number)) ]

        if invalid_types:
            raise TypeError(f"Incorrect types: {invalid_types}.")

        calc_cells = plot_cells.data[~(np.isnan(plot_cells.data))]

        # Compute vmin_cells and vmax_cells according the method
        robust_vmax_cells, robust_vmin_cells = None, None 
        if robust_cells:
            if robust_type == 'percentile':
                robust_vmax_cells = np.percentile(calc_cells, 5)
                robust_vmin_cells = np.percentile(calc_cells, 95)
            
            if robust_type == 'boundary':
                robust_vmax_cells = calc_cells.min()
                robust_vmin_cells = calc_cells.max()
        
        self.vmax_cells = robust_vmax_cells or self.DEFAULT_VMAX_CELLS
        self.vmin_cells = robust_vmin_cells or self.DEFAULT_VMIN_CELLS

        # Normalize the values and format into a unique type with the right imputation
        normalize = lambda x: _normalize_cell_size(
                                         size=x,
                                         minimum=self.vmin_cells,
                                         maximum=self.vmax_cells,
                                         alias_true=self.vmax_cells,
                                         alias_false=self.vmin_cells,
                                         alias_nan=0.0
                                        )                             

        plot_cells = np.ma.masked_array(data=[ list(map(normalize, l)) for l in plot_cells.data ],
                                        mask=plot_cells.mask)

        # Store the values
        self.plot_cells = plot_cells

    def _annotate_and_size_cells(self, ax, mesh, square_shaped_cells):
        """Add textual labels with the value in each cell."""
        # ( MODIFY: former _annotate_heatmap )
        mesh.update_scalarmappable()
        annot_data = self.annot_data or np.full(self.plot_data.shape, np.nan)
        height, width = self.plot_data.shape
        xpos, ypos = np.meshgrid(np.arange(width) + .5, np.arange(height) + .5)
        for x, y, m, color, annotation, cell_size in zip(xpos.flat, ypos.flat,
                                                  mesh.get_array(), mesh.get_facecolors(),
                                                  annot_data.flat, self.plot_cells.flat):
            if m is not np.ma.masked:
                size = np.clip ( cell_size / self.vmax_cells, 0.1, 1.0)
                shape = None
                if square_shaped_cells:
                    shape = plt.Rectangle((x - size / 2, y - size / 2),
                                          size,
                                          size,
                                          facecolor=color,
                                          **self.shape_kws)
                else:
                    shape = plt.Circle((x - size / 2, y - size / 2),
                                       size,
                                       facecolor=color,
                                       fill=True,
                                       **self.shape_kws)

                ax.add_patch(shape)

                if self.annot and not np.isnan(annotation):
                    lum = relative_luminance(color)
                    text_color = ".15" if lum > .408 else "w"
                    annotation = ("{:" + self.fmt + "}").format(annotation)
                    text_kwargs = dict(
                        color=text_color, ha="center", va="center")
                    text_kwargs.update(self.annot_kws)
                    ax.text(x, y, annotation, **text_kwargs)

    def _skip_ticks(self, labels, tickevery):
        """Return ticks and labels at evenly spaced intervals."""
        n = len(labels)
        if tickevery == 0:
            ticks, labels = [], []
        elif tickevery == 1:
            ticks, labels = np.arange(n) + .5, labels
        else:
            start, end, step = 0, n, tickevery
            ticks = np.arange(start, end, step) + .5
            labels = labels[start:end:step]
        return ticks, labels

    def _auto_ticks(self, ax, labels, axis):
        """Determine ticks and ticklabels that minimize overlap."""
        transform = ax.figure.dpi_scale_trans.inverted()
        bbox = ax.get_window_extent().transformed(transform)
        size = [bbox.width, bbox.height][axis]
        axis = [ax.xaxis, ax.yaxis][axis]
        tick, = axis.set_ticks([0])
        fontsize = tick.label1.get_size()
        max_ticks = int(size // (fontsize / 72))
        if max_ticks < 1:
            return [], []
        tick_every = len(labels) // max_ticks + 1
        tick_every = 1 if tick_every == 0 else tick_every
        ticks, labels = self._skip_ticks(labels, tick_every)
        return ticks, labels

    def plot(self, ax, cax, square_shaped_cells, **kwargs):
        """Draw the heatmap on the provided Axes."""
        # Remove all the Axes spines
        despine(ax=ax, left=True, bottom=True)

        # Draw the heatmap
        mesh = ax.pcolormesh(self.plot_data, vmin=self.vmin, vmax=self.vmax,
                             cmap=self.cmap, **kwargs)

        # Set the axis limits
        ax.set(xlim=(0, self.data.shape[1]), ylim=(0, self.data.shape[0]))

        # Invert the y axis to show the plot in matrix form
        ax.invert_yaxis()

        # Possibly add a colorbar
        if self.cbar:
            cb = ax.figure.colorbar(mesh, cax, ax, **self.cbar_kws)
            cb.outline.set_linewidth(0)
            # If rasterized is passed to pcolormesh, also rasterize the
            # colorbar to avoid white lines on the PDF rendering
            if kwargs.get('rasterized', False):
                cb.solids.set_rasterized(True)

        # Add row and column labels
        if isinstance(self.xticks, string_types) and self.xticks == "auto":
            xticks, xticklabels = self._auto_ticks(ax, self.xticklabels, 0)
        else:
            xticks, xticklabels = self.xticks, self.xticklabels

        if isinstance(self.yticks, string_types) and self.yticks == "auto":
            yticks, yticklabels = self._auto_ticks(ax, self.yticklabels, 1)
        else:
            yticks, yticklabels = self.yticks, self.yticklabels

        ax.set(xticks=xticks, yticks=yticks)
        xtl = ax.set_xticklabels(xticklabels)
        ytl = ax.set_yticklabels(yticklabels, rotation="vertical")

        # Possibly rotate them if they overlap
        if hasattr(ax.figure.canvas, "get_renderer"):
            ax.figure.draw(ax.figure.canvas.get_renderer())
        if axis_ticklabels_overlap(xtl):
            plt.setp(xtl, rotation="vertical")
        if axis_ticklabels_overlap(ytl):
            plt.setp(ytl, rotation="horizontal")

        # Add the axis labels
        ax.set(xlabel=self.xlabel, ylabel=self.ylabel)

        # Annotate the cells with the formatted values
        self._annotate_and_size_cells(ax, mesh, square_shaped_cells)


def custom_cells_heatmap(data, vmin=None, vmax=None, cmap=None, center=None, robust=False,
                        annot=None, fmt=".2g", annot_kws=None,
                        linewidths=0, linecolor="white",
                        cbar=True, cbar_kws=None, cbar_ax=None,
                        square=False, xticklabels="auto", yticklabels="auto",
                        mask=None, ax=None, data_cells=None, robust_cells=True,
                        robust_type='percentile', vmin_cells=None, vmax_cells=None,
                        shape_kws=None, normalize_cells=True,
                        square_shaped_cells=True, **kwargs):
    """ NEW-- Tweak the function heatmap by adding a third variable."""
    # Initialize the plotter object
    plotter =  _CustomCellHeatMapper(data, vmin, vmax, cmap, center,
                robust, annot, fmt, annot_kws, cbar, cbar_kws, shape_kws,
                data_cells, vmin_cells, vmax_cells, robust_cells,
                robust_type, xticklabels, yticklabels, mask,
                normalize_cells)

    # Add the pcolormesh kwargs here
    kwargs["linewidths"] = linewidths
    kwargs["edgecolor"] = linecolor

    # Draw the plot and return the Axes
    if ax is None:
        ax = plt.gca()
    if square:
        ax.set_aspect("equal")

    # delete grid
    # ax.grid(False)   

    plotter.plot(ax, cbar_ax, square_shaped_cells, **kwargs)
    return ax
